In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import re
import pandas as pd
import openai
# import os
import json
# from gpt3_sandbox.api.gpt import GPT
# from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
from GptPrompter import *
from GptCOTPrompter import *
from AutoReasoner import *
import dotenv

config = dotenv.dotenv_values(".env")
openai.api_type = 'azure'
openai.api_base = 'https://meta-prompter-az-openai.openai.azure.com'
openai.api_version = '2022-12-01'
openai.api_key = config['OPENAI_API_KEY_ms']

dataset = pd.read_csv('./dataset/FeTaQA/test.csv', sep=',')
# dataset = pd.read_csv('./dataset/FeTaQA/test_sample400.csv', sep=',')

ft = None

In [4]:
# sampled_df = dataset.sample(frac=0.2, random_state=0)
# print(sampled_df.shape)
# sampled_df.to_csv('./dataset/FeTaQA/test_sample400.csv', index=False)

In [4]:
# import fasttext
# ft = fasttext.load_model('cc.en.300.bin')

In [29]:
NNDemo = False
max_demo = 5

# template = 'original-sql-py'
template = 'original-sql-py-no-intermediate'
# template = 'formatv1-sql-py'

# gpt_model = 'text-davinci-003'
# gpt_model = 'gpt-3.5-turbo'
gpt_model = 'mp-aoi-codex'

def parallel_codex_func_formatv1(i):
    max_retry = 3
    while max_retry>0:
        try:
            codex_prompter = CodexAnswerCOTExecutor_template(
                                      f'prompt_template/{template}.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'],  
                                      base_path='./dataset/FeTaQA/',
                                      demo_file=f'few-shot-demo/FeTaQA-{program}.json',
                                     )
            codex_prompter.model = gpt_model
            codex_prompter.max_demo = max_demo
            # codex_prompter.demo_ids = [0, 1, 2, 3, 6, 8, 11]
            
            # codex_prompter._gen_gpt_prompt()
            codex_prompter._gen_gpt_prompt(NNDemo, ft)
            codex_prompter._get_gpt_prediction()
            log = codex_prompter._log_dict()
            break
        except Exception as e:
            log = {
                'id': dataset.iloc[i]['id'],
                'uncaught_err': str(e)
            }
            if "model's maximum context length" in str(e):
                return log
            max_retry -= 1
    return log
    
for program in ['sql-py']:
    n_threads = 2
    maxLimit = float('inf')
    # maxLimit = 100
    from joblib import Parallel, delayed
    output_result_file = f'./dataset/FeTaQA/results/CodexAnswerCOTExecutor_{template}_{program}_NNDemo={NNDemo}_results_pristine-unseen-tables_limit{maxLimit}_model{gpt_model}.json'
    logs = Parallel(n_jobs=n_threads, require='sharedmem')(delayed(parallel_codex_func_formatv1)(i) for i in tqdm(range(min(maxLimit, dataset.shape[0]))))
    json.dump(logs, open(output_result_file, 'w'), indent=4)
    # evaluate: 

 10%|████████████████████████                                                                                                                                                                                                                    | 204/2006 [03:21<26:03,  1.15it/s]

Error: string indices must be integers


 39%|████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                               | 788/2006 [12:52<21:22,  1.05s/it]

Connecting to SQLite for execution code: SELECT ABS(111.688 - 111.470) AS diff FROM DF;.


 89%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                          | 1778/2006 [29:34<03:36,  1.05it/s]

Error: string indices must be integers


 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊        | 1936/2006 [32:06<01:21,  1.17s/it]

Connecting to SQLite for execution code: SELECT ROUND(votes, 1) FROM DF;.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2006/2006 [33:15<00:00,  1.01it/s]


In [47]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import nltk
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
BLEUs = []

all_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
BLEU = 0
for l in logs:
    if not 'predicted_value' in l:
        continue
    # scores = scorer.score(l['target_value'], l['predicted_value'])
    scores = scorer.score(l['target_value'], l['utterance'] + ' ' + l['predicted_value'])
    # scores = scorer.score(l['predicted_value'], l['target_value'])
    
    # print(scores)
    for m in scores:
        all_scores[m].append(scores[m].recall)
        # all_scores[m].append(scores[m].precision)
    
#     # candidate_tokens = nltk.word_tokenize(l['predicted_value'].lower())
#     # reference_tokens = nltk.word_tokenize(l['target_value'].lower())
#     candidate_tokens = l['predicted_value'].split()
#     reference_tokens = l['target_value'].split()
    
#     BLEUs.append(nltk.translate.bleu_score.sentence_bleu([reference_tokens], candidate_tokens))
    
    # break
all_scores = {'rouge1': np.mean(all_scores['rouge1']), 'rouge2': np.mean(all_scores['rouge2']), 'rougeL': np.mean(all_scores['rougeL'])}
all_scores

{'rouge1': 0.6473680302800984,
 'rouge2': 0.41243101776713587,
 'rougeL': 0.5450562792752759}

In [62]:
import nltk

def calculate_bleu_score(predicted_text, reference_text):
    # print(predicted_text)
    # print(reference_text)
    smoothie = nltk.translate.bleu_score.SmoothingFunction().method4
    bleu_score = nltk.translate.bleu_score.sentence_bleu(
        [reference_text.split()],                                                  
        predicted_text.split(),
        smoothing_function=smoothie
    )
    # print(bleu_score)
    return bleu_score

BLEU = 0
BLEUs = []
for l in logs:
    if not 'predicted_value' in l:
        continue
    
    BLEUs.append(calculate_bleu_score(l['predicted_value'], l['target_value']))
    
print("BLEU Score:", np.average(BLEUs))

BLEU Score: 0.20958185489849235


In [54]:
reference_corpus

NameError: name 'reference_corpus' is not defined

In [13]:
NNDemo = False
max_demo = 5
template = 'original-sql-py-no-intermediate'
program = 'sql-py'
gpt_model = 'mp-aoi-codex'


def func(i):
    codex_prompter = CodexAnswerCOTExecutor_template(
                                      f'prompt_template/{template}.json',
                                      dataset.iloc[i]['id'], 
                                      dataset.iloc[i]['utterance'], 
                                      dataset.iloc[i]['context'], 
                                      dataset.iloc[i]['targetValue'],  
                                      base_path='./dataset/FeTaQA/',
                                      demo_file=f'few-shot-demo/FeTaQA-{program}.json',
                                     )
    codex_prompter.model = gpt_model
    codex_prompter.max_demo = max_demo
    # codex_prompter.demo_ids = [0, 2, 8, 11]

    # codex_prompter._gen_gpt_prompt()
    codex_prompter._gen_gpt_prompt(NNDemo, ft)
    codex_prompter._get_gpt_prediction()
    log = codex_prompter._log_dict()
    # print(vars(codex_prompter))
    # print(codex_prompter.frequency_penalty)
    return log
a = func(1)

In [14]:
# print(a['prompt'])
a

{'id': 15555.0,
 'utterance': 'What were the voting results of the 1975 North Vietnamese legislative election with regards to seats?',
 'source_csv': './dataset/FeTaQA/totto_source/train_json/example-7854.csv',
 'target_value': 'In the 1975 North Vietnamese legislative election, the Vietnamese Fatherland Front won all 424 seats.',
 'predicted_value': 'The Vietnamese Fatherland Front won 424 seats.',
 'prompt': 'The database table DF is shown as follows:\n[HEAD]: aircraft|introduced|retired\n---\n[ROW] 1: Aérospatiale N 262|1967|1969\n[ROW] 2: Boeing 707|-|-\n[ROW] 3: Boeing 720|1976|1977\n[ROW] 4: Convair 990 Coronado|1974|1975\n[ROW] 5: de Havilland Comet|1962|1965\n[ROW] 6: Douglas DC-3|1947|1976\n[ROW] 7: Douglas DC-4|1949|1953\n[ROW] 8: Douglas DC-8|1972|1978\n...\n[ROW] 14: Sud Aviation Caravelle|-|-\n[ROW] 15: Vickers VC10|1965/1977|1971/1978\n\nAnswer the following question based on the data above: "What Hawker planes has Air Ceylon had?". Generate SQL or Python code step-by-ste

In [64]:
import rouge_score 

In [65]:
rouge_score.__version__

AttributeError: module 'rouge_score' has no attribute '__version__'